In [19]:
hints = "spacy"

In [2]:
import pandas as pd
import numpy as np
import re

In [6]:
import os
os.chdir("..")

In [18]:
def get_spacy_hints(df, nlp):
    """extract noun chunks with relevant entity types and prepend them to text"""
    new_text = []
    for row in df.iterrows():
        ents = []
        doc = nlp(row[1]["text"])
        new = row[1]["text"]
        for chunk in doc.noun_chunks:
            #check if any words of the chunk are relevant entities 
            if set([word.ent_type_ for word in chunk]) & set(["GPE", "NORP", "EVENTS", "FAC", "LAW", "ORG", "PERSON"]) != set():
                new = new.replace(chunk.text, "<ents> " + chunk.text + " </ents>")

        new_text.append([new, row[1]["triplets"]])
    print(new)
    return pd.DataFrame(new_text, columns = ["text","triplets"])

In [16]:
def get_gold_hints(df):
    new_text = []
    for row in df.iterrows():
        
        subj = re.findall("(?<=<triplet> ).*?(?= <subj>)", row[1]["triplets"])
        obj = re.findall("(?<=<subj> ).*?(?= <obj>)", row[1]["triplets"])

        ents = subj + obj
        ents = list(set(ents))

        new = row[1]["text"]
        for ent in ents:
            new = new.replace(ent, "<ents> " + ent + " </ents>")

        #add identified entities between entitiy tokens before text
        #change token depending on ConfliBERT / REBEL
        #new = "<ents> " + ", ".join(list(set(ents))) + " </ents> " + row[1]["text"]
        new_text.append([new, row[1]["triplets"]])
        print(ents)
        print(new)
        print("---")
    return pd.DataFrame(new_text, columns = ["text","triplets"])

In [20]:
train = pd.read_csv(f"data_src/annotated/new_train_aug_0.csv", index_col = 0)
train = train.rename(columns = {"label":"triplets"})
val = pd.read_csv(f"data_src/annotated/new_val_aug_0.csv", index_col = 0)
val = val.rename(columns = {"label":"triplets"})
test = pd.read_csv(f"data_src/annotated/new_test_aug_0.csv", index_col = 0)
test = test.rename(columns = {"label":"triplets"})

if hints == "spacy":
    import spacy 
    nlp = spacy.load("en_core_web_trf")

    train = get_spacy_hints(train, nlp)
    val = get_spacy_hints(val, nlp)
    test = get_spacy_hints(test, nlp)

elif hints == "gold":
    
    train = get_gold_hints(train.iloc[:200])
    val = get_gold_hints(val)
    test = get_gold_hints(test)


/home/werner/thesis_valentin/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<ents> Liberians </ents> Fortify Lines of Defense in <ents> the Khartoum government </ents>.
<ents> Serb forces </ents> killed <ents> three ethnic Albanians </ents> in a gunbattle in <ents> southeastern Serbia </ents> Friday
<ents> North Korea </ents> has trained more than 500 computer hackers capable of launching cyber warfare against <ents> the United States </ents>
